In [1]:
from tools.transcripts import *
from tools.psl import *
import pandas as pd
from tools.mathOps import *
from tools.sqlInterface import *
from tools.nameConversions import *
%matplotlib inline
import seaborn as sns
import pysam
from glob import glob
import os

In [39]:
new_dir = '/public/groups/cgl/cat/primates_evan/work/transMap'
old_dir = '/public/groups/cgl/cat/primates_evan/original_primates/work/transMap/'
genome = 'Rhesus'

In [3]:
ref_df = load_annotation('/public/groups/cgl/cat/primates_evan/out/databases/Human.db')

In [42]:
def load_metrics(psl_dict):
    m = [[strip_alignment_numbers(x.q_name), f'{x.t_name}:{x.t_start}-{x.t_end}', x.coverage, x.identity] for x in psl_dict.values()]
    return pd.DataFrame(m, columns=['TranscriptId', 'Position', 'Coverage', 'Identity'])

gene_sets = []

new_psl = get_alignment_dict(f'{new_dir}/{genome}.filtered.psl')
old_psl = get_alignment_dict(f'{old_dir}/{genome}.filtered.psl')
new_metrics_df = load_metrics(new_psl)
old_metrics_df = load_metrics(old_psl)
merged = new_metrics_df.merge(old_metrics_df, on='TranscriptId', 
                              suffixes=['_Mmul_10', '_Mmul_8.0.1'], how='outer')
df = merged

In [43]:
df = df.merge(ref_df[['TranscriptId', 'TranscriptBiotype']], on='TranscriptId')

In [46]:
r = []
for biotype, s in df.groupby('TranscriptBiotype'):
    missing_new = len(s[s['Coverage_Mmul_10'].isnull()])
    missing_old = len(s[s['Coverage_Mmul_8.0.1'].isnull()])
    tot = len(s)
    s = s.dropna()
    average_cov_change = (s['Coverage_Mmul_10'] - s['Coverage_Mmul_8.0.1']).mean()
    r.append([biotype, tot, missing_new, missing_old, average_cov_change])

In [47]:
rdf = pd.DataFrame(r, columns=['biotype', 'num_in_gencode', 'missing_new', 'missing_old', 'avg_cov_change'])

In [48]:
rdf[rdf.biotype.str.contains('RNA')].to_csv('ncrna.csv')

In [49]:
q=rdf[rdf.biotype.str.contains('RNA')]
q['delta'] = q['missing_new'] - q['missing_old']

/public/home/ifiddes/anaconda2/envs/cat3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
q

,biotype,num_in_gencode,missing_new,missing_old,avg_cov_change,delta
7,Mt_tRNA,8,0,8,NaN,-8
15,lncRNA,44042,689,1420,0.003738,-731
16,miRNA,1809,23,75,0.005891,-52
17,misc_RNA,2106,33,64,0.011977,-31
25,rRNA,28,0,8,0.018268,-8
26,rRNA_pseudogene,441,7,14,0.006534,-7
29,sRNA,3,0,0,-0.005747,0
30,scRNA,1,0,0,-0.015000,0
31,scaRNA,46,2,0,-0.001991,2
32,snRNA,1714,45,98,0.000625,-53


In [51]:
filt_df = df[df.TranscriptBiotype.str.contains('RNA')]

In [55]:
filt_df.to_csv('ncrna_full_table_mmul.csv')